In [ ]:
from pyhive import hive
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
#%matplotlib inline
%matplotlib notebook
import random
from itertools import count
import numpy as np
from datetime import datetime
from pandas._libs.tslibs.timestamps import Timestamp

In [ ]:
from common_vars import *

In [ ]:
hive_cnx = hive.Connection(
  host = hdfs_host, 
  port = hive_port, 
  username = hive_username,
  password = hive_password,
  auth = hive_mode,
  database=hive_database)

In [ ]:
def run_queries(queries):
    with hive_cnx.cursor() as cursor:   
        for q in queries:
            cursor.execute(q)    
        if cursor.poll().hasResultSet:
            return cursor.fetchall()


In [ ]:
tweet_keys = ['id', 'text','created_at', 'geo', 'coordinates', 'place',
              'quote_count', 'reply_count', 'retweet_count', 'favorite_count',
              'user_id', 'user_followers','Sentiment','n_words', 'key']

cols = ', '.join(tweet_keys)

q = [f"select {cols} from tweets where user_followers is not NULL"]
q

In [ ]:
tweets = run_queries(q)

In [ ]:
len(tweets)

In [ ]:
tweetsDF = pd.DataFrame(tweets, columns= tweet_keys)
tweetsDF.describe()

In [ ]:
plt.figure(figsize=(13,9))
plt.scatter(tweetsDF['user_followers'], tweetsDF['Sentiment'] )


In [ ]:

t = tweetsDF[['created_at','user_followers','Sentiment']]

log10  = lambda x : np.log10(x +1)

t.loc[:,'user_followers'] = tweetsDF['user_followers'].apply(log10)

timestp = lambda x: datetime.utcfromtimestamp(x.timestamp()) 

t.loc[:,'created_at'] = tweetsDF['created_at'].apply(timestp)


In [ ]:
fig = plt.figure(figsize=(13,9))
plt.scatter(t['user_followers'], t['Sentiment'])
plt.xlabel('log10(user_followers)')
plt.ylabel('Sentiment')

In [ ]:
plt.figure(figsize=[13,9])
hst = plt.hist2d(t.user_followers[t.Sentiment!=0], t.Sentiment[t.Sentiment!=0], bins=[100,100])

In [ ]:
q = .7
qS = abs(t.Sentiment).quantile(q)
qS

In [ ]:
plt.figure(figsize=[13,9])
plt.hist(t.Sentiment[t.Sentiment!=0], bins=200)
plt.vlines([-qS,qS],ymin= 0, ymax = 10,colors='red')

In [ ]:
quf = t.user_followers.quantile(q)
quf

In [ ]:
plt.figure(figsize=[9,9])
plt.hist(t.user_followers[t.Sentiment!=0], bins=200)
plt.vlines([quf],ymin= 0, ymax = 5,colors='red')

In [ ]:
#tt = (t.user_followers>quf) & (t.user_followers<5) & ((t.Sentiment) < -qS)
tt =  ((t.Sentiment) < -qS)


In [ ]:
tt = tweetsDF[tt]
len(tt)

In [ ]:
badboys = tt.sort_values(by=['Sentiment','user_followers'], ascending = [True, False] )
cols = "text created_at user_followers Sentiment".split(" ")

print("\n-----------------------------\n".join(badboys['text']))

In [ ]:
fig = plt.figure(figsize=(13,9))
plt.scatter(np.log10(badboys['user_followers']), badboys['Sentiment'],)
plt.xlabel('log10(user_followers)')
plt.ylabel('Sentiment')

In [ ]:
badboys.index = range(len(badboys))
i=0

In [ ]:
# print(badboys.text[i])
# i+=1

In [ ]:
# a,b = min(t['created_at']), max(t['created_at'])
# a,b

In [ ]:
from pyhive import hive
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import dates as mdates

from itertools import count
import numpy as np
%matplotlib notebook
n = len(t)

plt.style.use('fivethirtyeight')


# Parameters
x_len = 200       # Number of points to display
y_range = [-5,5]  # Range of possible Y values to display

# Create figure for plotting
fig = plt.figure(figsize = [9,9])
ax = fig.add_subplot(1, 1, 1)
xs = list(range(0, 200))
ys = [0] * x_len
ax.set_ylim([-100,100])
#ax.set_xlim([a.timestamp(),b.timestamp()])

ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.xaxis_date()
x = []
y1 = []
y2 = []
wtf = []
#index = count()

#plt.plot([], [], label='Channel 1')
#plt.plot([], [], label='Channel 2')


def animate(i):
    data = t[:i]
    x = data['created_at']
    y1 = data['user_followers']
    y2 = data['Sentiment']
    
    #wtf.append(ax.xlim)
    
    ax.clear()
    
    #ax.scatter(x,y1, label='user_followers')
    ax.scatter(x,y2, label='Sentiment')
    ax.set_xlim([a,b])
    plt.xticks(rotation=45, ha='right')
    plt.legend(loc='upper left')
    plt.subplots_adjust(bottom=0.30)
    #plt.xlim = [0,24]
#     plt.ylim = [-100,100]    
#     plt.tight_layout()


ani = FuncAnimation(fig, animate, frames=range(4000,4800), interval=100)#4000

plt.tight_layout()
plt.show()

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=[13,9])
ax = fig.add_subplot(1, 1, 1)
xs = list(range(0, 200))
ys = [0] * x_len
ax.set_ylim([-100,100])
ax.set_xlim([a,b])
animate(4800)
